# <font size=8 color=steelblue >**Phase 3: Problem Statement — RetailSmart Advanced Analytics**
------------------
-----------------


# <font size=5 color=lightseagreen >Business Context
---

RetailSmart is an omni-channel e-commerce company that has already built strong capabilities in data cleaning, exploratory analysis, and churn prediction.
While the predictive model (from Phase 2) helps the company identify which customers are likely to churn, senior management now wants to deepen analytical intelligence in three critical areas:  

- **Customer Segmentation** — to personalize campaigns and retention offers
- **Demand Forecasting** — to plan production, procurement, and logistics
- **Cross-Sell Recommendations** — to increase average order value through bundled sales

The goal of this phase is to use unsupervised learning, time-series analysis, and association rule mining to discover patterns and trends that cannot be easily captured through supervised modeling.

1. <font color=skyblue >Customer Segmentation (Unsupervised Learning)

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

- **Group customers into meaningful segments using behavioral and marketing features
(e.g., Recency, Frequency, Monetary, Avg Spend, Response Rate).**

------

- **Identify cluster profiles such as High-Value Loyalists, Price-Sensitive Frequent Buyers,
or At-Risk Customers.**

---------

- **Visualize segment separation using PCA and summarize behavioral differences
between clusters.**

---

- **Generate and save a summary of customer clusters as
data_outputs/cluster_summary.csv.**

---

2. <font color=skyblue >Demand Forecasting (Time Series Analysis)

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

- **Analyze historical order and revenue trends.**

----

- **Build a forecasting model to predict order volumes and total revenue for the next 6
months.**

---

- **Provide insights on seasonality and growth trends for operational planning.**

---

- **Save actual vs. forecasted monthly order volumes as
data_outputs/forecast_results.csv.**

-----

- **Save the top product-pair associations (support, confidence, lift) as
data_outputs/association_rules.csv.**

---